In [ ]:
#%matplotlib notebook
%matplotlib inline
from matplotlib import rc
rc('figure',figsize=(16,8))
rc('font',size=12)
rc('text',usetex=False)

from qubicpack import qubicpack as qp
import fibtools as ft
import plotters as p
import lin_lib as ll
import demodulation_lib as dl
import qubic

from pysimulators import FitsArray

import numpy as np
from matplotlib.pyplot import *
import matplotlib.mlab as mlab
import scipy.ndimage.filters as f
import glob
import string
import scipy.signal as scsig
from scipy import interpolate
import os
import healpy as hp

In [ ]:
dirallfreqs = '/Users/hamilton/Google Drive/QUBIC/Calib-TD/Files/Synthesized Beams/Synthesized_Beams_Files/'
dirfreq = ['130GHz-2019-04-18/', '140GHz-2019-04-10/', '150GHz-2019-04-06/', '160GHz-2019-04-08/', '170GHz-2019-04-14/']
freqs = np.array([130, 140, 150, 160, 170])

In [ ]:
reload(dl)
ifreq = 1
img, az, el = dl.get_flatmap(93, dirallfreqs+dirfreq[ifreq])

def disp(img, az, el, nsmin=2, nsmax=40, vmin = None, vmax = None):
    mm, ss = ft.meancut(img, 3)
    if vmin is None:
        vmin = mm-nsmin*ss
    if vmax is None:
        vmax = mm+nsmax*ss
    imshow(img, extent=[np.min(az)*np.cos(np.radians(50)), np.max(az)*np.cos(np.radians(50)), np.min(el), np.max(el)],
       aspect='equal', vmin=vmin, vmax=vmax)
    xlabel(r'$Az \times \cos(50)$')
    ylabel(r'$Elevation$')
    title('Frequency: {} GHz'.format(freqs[ifreq]))
    colorbar()

disp(img, az, el)
mm, ss = ft.meancut(img, 3)

In [ ]:
nbl = len(el)
nbaz = len(az)
nperaz = 1
daz = nbaz/nperaz
ifile = 0
theimg = np.zeros_like(img)
for i in xrange(len(el)):
    clf()
    theimg[i,:] += img[i,:]
    disp(theimg, az, el, vmin=mm-2*ss, vmax=mm+40*ss)
    savefig('/Volumes/Data/Qubic/Calib-TD/Animations/scan_150GHz/imgscan_{0:04d}.png'.format(ifile))
    ifile += 1

In [ ]:
reload(dl)
for ifreq in xrange(len(freqs)):
    img = dl.get_hpmap(93, dirallfreqs+dirfreq[ifreq])
    clf()
    mm, ss = ft.meancut(img[img != 0],3)
    hp.gnomview(img, reso=10, min=mm-2*ss, max=mm+60*ss, title='Frequency: {} GHz'.format(freqs[ifreq]))
    savefig('/Volumes/Data/Qubic/Calib-TD/Animations/all_freqs/imgscan_{0:04d}.png'.format(freqs[ifreq]))

In [ ]:
from __future__ import division
import healpy as hp
import numpy as np
import matplotlib.pyplot as mp
from pysimulators import FitsArray
import time
import qubic

dictfilename = '/Users/hamilton/testTD.dict'

d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
d['nside'] = 256
d['filter_relative_bandwidth'] = 0.35


for ff in freqs:
    d['filter_nu'] = ff*1e9

    # Create scene
    s = qubic.QubicScene(d)

    # Create instrument model
    inst = qubic.QubicInstrument(d)
    sb = inst[180].get_synthbeam(s)[0]
    clf()
    hp.gnomview(sb/np.max(sb), reso=10, rot=[0,90],min=0, max=0.15,cmap='viridis',title='Frequency = {} GHz'.format(ff))
    savefig('/Volumes/Data/Qubic/Calib-TD/Animations/all_freqs/imgTrue_{0:04d}.png'.format(ff))

In [ ]:
reload(dl)
for ifreq in xrange(len(freqs)):
    img = dl.get_hpmap(93, dirallfreqs+dirfreq[ifreq])
    clf()
    mm, ss = ft.meancut(img[img != 0],3)
    hp.gnomview(img, reso=10, min=mm-1*ss, max=mm+60*ss, title='Frequency: {} GHz - Data'.format(freqs[ifreq]),
               sub=(1,2,1))
    
    ff = freqs[ifreq]
    d['filter_nu'] = ff*1e9
    # Create scene
    s = qubic.QubicScene(d)
    # Create instrument model
    inst = qubic.QubicInstrument(d)
    sb = inst[180].get_synthbeam(s)[0]
    hp.gnomview(sb/np.max(sb), reso=10, rot=[0,90],min=0, max=0.15,cmap='viridis',
                title='Frequency = {} GHz - Theory'.format(ff), sub=(1,2,2))
    
    savefig('/Volumes/Data/Qubic/Calib-TD/Animations/all_freqs/imgscan_and_True_{0:04d}.png'.format(freqs[ifreq]))
    